In [1]:
import os
import json
import pandas as pd
from datetime import datetime
import requests
from collections import Counter
from konlpy.tag import Okt

# JSON 파일 병합 및 필터링
folder_path = "./"  # 동일 폴더
start_date = "2024-11-02"
end_date = "2024-11-12"

# JSON 병합
all_data = []
for file_name in os.listdir(folder_path):
    if file_name.endswith(".json"):
        with open(os.path.join(folder_path, file_name), "r", encoding="utf-8") as file:
            data = json.load(file)
            for post in data:
                date_key = next((key for key in ["post_date", "작성일", "date"] if key in post), None)
                if date_key:
                    post_date = post[date_key]
                    all_data.append({
                        "date": post_date,
                        "content": post.get("content", ""),
                        "comments": post.get("comments", ""),
                        "title": post.get("title", ""),
                        "community": post.get("community", file_name.split(".json")[0])
                    })

# 데이터프레임 생성
df = pd.DataFrame(all_data)
df["date"] = pd.to_datetime(df["date"], errors="coerce")
df = df.dropna(subset=["date"])  # 날짜 파싱 실패 행 제거
df = df[(df["date"] >= start_date) & (df["date"] <= end_date)]

In [15]:
# 불용어 리스트 로드
stopwords_url = "https://raw.githubusercontent.com/stopwords-iso/stopwords-ko/master/stopwords-ko.json"
response = requests.get(stopwords_url)
STOPWORDS = set(response.json())
STOPWORDS.update(['지금', '우리', '그때', '처음', '보고', '삭제', '진짜', '다시'])

# 형태소 분석기 초기화 (Okt)
okt = Okt()

In [16]:
# 명사 및 고유명사 추출 함수 (최적화)
def extract_nouns_optimized(text, stopwords):
    if not text:  # 빈 텍스트 처리
        return []
    pos_tags = okt.pos(text, norm=True, stem=True)  # 표준화 및 어간 추출
    return [
        word for word, tag in pos_tags
        if tag in ['Noun', 'ProperNoun'] and word not in stopwords and len(word) > 1
    ]  # 필터링 단계 간소화

# 데이터프레임의 모든 텍스트 병합 및 명사 추출
def process_dataframe(df, stopwords):
    word_data = []
    for index, row in df.iterrows():
        combined_text = f"{row['content']} {row['comments']} {row['title']}"
        nouns = extract_nouns_optimized(combined_text, stopwords)
        word_data.extend([(noun, row["date"].strftime("%Y-%m-%d"), row["community"]) for noun in nouns])
    return word_data

# 단어별 집계 및 결과 정리
def generate_results(word_data):
    word_counter = Counter([word[0] for word in word_data])
    result = [
        {
            "word": word,
            "count": count,
            "dates": list(set(d[1] for d in word_data if d[0] == word)),
            "communities": list(set(d[2] for d in word_data if d[0] == word))
        }
        for word, count in word_counter.most_common()
    ]
    return result

# 불용어 리스트와 데이터 처리
word_data = process_dataframe(df, STOPWORDS)
result = generate_results(word_data)

# 결과 저장
output_file = "word_analysis_v2.json"
with open(output_file, "w", encoding="utf-8") as file:
    json.dump(result, file, ensure_ascii=False, indent=4)

print(f"결과가 {output_file}에 저장되었습니다.")


결과가 word_analysis_v2.json에 저장되었습니다.


In [ ]:
import json

# word_analysis.json 파일에서 단어만 출력
output_file = "word_analysis_v2.json"

with open(output_file, "r", encoding="utf-8") as file:
    data = json.load(file)

# 단어만 추출하여 출력
words = [entry["word"] for entry in data]
words_30th = words[:30]
print("추출된 단어 목록:")
print(words_30th)

추출된 단어 목록:
['사람', '댓글', '새끼', '일시', '그냥', '생각', '한국', '존나', '여자', '병신', '미국', '정도', '남자', '소리', '문제', '나라', '트럼프', '지랄', '저런', '중국', '때문', '이제', '일본', '수준', '시발', '씨발', '제발', '요즘', '자체', '회사']


In [ ]:
from konlpy.tag import Kkma
import pandas as pd

kkma = Kkma()

data = pd.read_json('word_analysis_v2.json')

data = data.loc[data['count'] >= 100, :]
data['pos'] = [kkma.pos(w) for w in data['word']]
data['NNP'] = data['pos'].apply(lambda x : "NNP" in str(x))
data.to_excel('고유명사.xlsx', index = False)
data.loc[data['NNP'] == True, :]